<div><img src="Images/SKI_Header_1_Tr.png" width="150%"/></div>
<p style="text-align: center; font-size: 36pt; font-weight: bold; color: rgb(0, 0, 190); padding: 20px; margin-top: 2%;">КУРС</p>
<p style="text-align: center; font-size: 36pt; font-weight: bold; color: rgb(0, 0, 190); padding: 40px; margin-top: -3%; line-height: 100%;">Архитектура ЭВМ и язык ассемблера</p>
<p style="text-align: center; font-size: 28pt; font-weight: bold; color: rgb(0, 0, 190); padding: 40px; margin-top: -3%; line-height: 100%;">СЕМИНАРЫ</p>
<p style="text-align: left; font-size: 20pt; font-weight: bold; color: rgb(0, 0, 190); padding: 20px; margin-top: -2%; line-height: 10%;">Преподаватель:</p>
<p style="text-align: left; font-size: 20pt; font-weight: bold; color: rgb(0, 0, 190); padding: 18px; margin-top: -2%; line-height: 100%;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Русол Андрей Владимирович</p>
<p style="text-align: left; font-size: 20pt; font-weight: bold; color: rgb(0, 0, 190); padding: 18px; margin-top: -2%; line-height: 100%;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;к.т.н., с.н.с. ГЕОХИ РАН</p>
<p style="text-align: center; font-size: 16pt; font-weight: bold; color: rgb(0, 0, 190); padding: 20px; margin-top: 2%;">Москва 2022</p>


<p style="text-align: center; font-size: 36pt; font-weight: bold; color: rgb(0, 0, 190); padding: 40px; margin-top: -3%; line-height: 100%;">Семинары № 5 и 6</p>


<p style="text-align: center; font-size: 28pt; font-weight: bold; color: rgb(0, 0, 190); padding: 40px; margin-top: -3%; line-height: 100%;">Циклы в языке Ассемблер</p>

В языке Assembler cуществует несколько способов организации циклического повторения фрагмента кода. Каждый из способов имеет свои особенности, поэтому для эффективной реализации кода в каждом конкретном случае следует использовать наиболее подходящий способ.

## Первый способ — организация цикла с помощью команд условного перехода и регистра ecx/cx .

В архитектуре микропроцессоров Intel регистр **ЕСХ/СХ** имеет определенное функциональное назначение — играет роль счетчика.

Если некоторую группу команд необходимо повторить определенное количество раз, цикл можно организовать следующим образом:

1. поместить в регистр **ЕСХ/СХ** количество повторений;

2. первую команду тела цикла отметить меткой;

3. после выполнения тела цикла уменьшить содержимое регистра **ЕСХ/СХ** на 1;

4. выполнить сравнение содержимого регистра **ЕСХ/СХ** нулем;

5. в случае, если **ЕСХ/СХ** не равно **0**, осуществить переход на метку, иначе выполнять следующую за телом цикла команду.

Реализации может выглядеть таким образом:
```asm
         mov ecx,N

   CYCL:

         <тело цикла>

         dec ecx

         cmp ecx,0

         jne CYCL

         ...
```

**Внимание:** необходимо отслеживать возможность выполнения «пустого» цикла. Если начальное значение в регистре **СХ** окажется равным нулю, то после уменьшения на единицу, содержимым регистра станет число **FFFFh** (-1 в десятичной системе счисления). В результате цикл повторится еще **65535 раз (или 4 294 967 295 раз при использовании регистра ecx)**. Поэтому следует всегда проверять содержимое регистра **ЕСХ/СХ** для предотвращения выполнения «пустого» цикла. 

#### Пример

Простой вывод строки в консоль.

```
%include "io.inc"

section .data  
str: db `Hello, World!\n`, 0


section .text
global CMAIN
CMAIN:
    PRINT_STRING [str]  
    mov eax, 0
    ret

```

Добавим переменную с **n** числом повторений, напишем цикл вывода строки.


```asm
%include "io.inc"

section .data  
n dd 10
str: db `Hello, World!\n`, 0

section .text
global CMAIN
CMAIN:
    mov ecx, [n]
CYCL:
    PRINT_STRING [str]  
    dec ecx
    cmp ecx,0
    jne CYCL
    
    mov eax, 0
    ret

```

Предотвращение длительного выполнения "пустых" циклов можно организовать с помощью команд условного перехода **JCXZ/JECXZ** - Jump if CX/ECX is Zero.

Синтаксис команды:

```asm
jcxz <метка перехода>
```

Либо в случае необходимости использования расширенного регистра

```asm
jecxz <метка перехода>
```

**ВНИМАНИЕ:** команда **JCXZ/JECXZ** может адресовать только короткие переходы (на –128 или на +127 байт от следующей за ней команды).  

#### Пример

```asm
%include "io.inc"

section .data  
n dd 0
str: db `Hello, World!\n`, 0

section .text
global CMAIN
CMAIN:
    mov ecx, [n]
    jecxz EXIT
CYCL:
    ;PRINT_STRING [str]  
    dec ecx
    cmp ecx,0
    jne CYCL
    
EXIT:
    mov eax, 0
    ret
```

## Второй способ — организация цикла с помощью команды безусловного перехода jmp и регистра **ЕСХ/СХ**

Цикл можно организовать и следующим образом:

1. поместить в регистр **ЕСХ/СХ** количество повторений;

2. осуществить проверку на «пустой» цикл командой **JCXZ/JECXZ**

3. эту команду проверки отметить меткой начала цикла;

4. после выполнения тела цикла уменьшить содержимое регистра **ЕСХ/СХ** на 1;

5. выполнить безусловный переход на начало цикла.

Схема реализации:

```asm
         mov cx,N

   CYCL:

         jcxz Exit

         <тело цикла>

         dec cx

         jmp CYCL

   Exit:    . . .
```

Здесь команда jcxz играет двойную роль: 
- во-первых, предотвращает выполнение «пустого» цикла, 
- во-вторых, отслеживает окончание цикла. 

#### Пример

```asm
%include "io.inc"

section .data  
n dd 10
str: db `Hello, World!\n`, 0

section .text
global CMAIN
CMAIN:
    mov ecx, [n]
    
CYCL:    
    jecxz EXIT
    ;PRINT_STRING [str]  
    dec ecx
    cmp ecx,0
    jmp CYCL
    
EXIT:
    mov eax, 0
    ret
```

## Третий способ — организация цикла с помощью специальных команд

В первых двух способах организации цикла большинство операций выполняются “вручную” (декремент регистра cx, сравнение cx с нулем, переход на начало цикла).

Учитывая важность циклов в системы команд микропроцессора введены специальные команды, облегчающие (сокращающие) программирование циклических участков программ

```asm
loop <метка перехода>
loope/loopz <метка перехода>
loopne/loopnz <метка перехода>
```

**Внимание**, эти команды также используют регистр **ЕСХ/СХ** как параметр (счетчик) цикла.

## Организация цикла с помощью команды loop

В переводе Loop означает «петля», или другими словами «повторить цикл».

Синтаксис команды:
```asm
loop <метка перехода>
```

Работа команды заключается в выполнении следующих действий:

1. уменьшение значения регистра **ЕСХ/СХ** на 1;
2. сравнение регистра **ЕСХ/СХ** с нулем.

Если **ЕСХ/СХ** оказывается равным **0**, то осуществляется выход из цикла, т.е. управление передается на следующую после loop команду, иначе — управление передается на метку перехода (цикл повторяется).

Порядок организации цикла с помощью команды **loop**:

1. в регистр cx поместить значение, равное количеству повторений;
2. установить метку на первую команду тела цикла;
3. выполнить команду ``loop <метка перехода>``.

Схема реализации выглядит следующим образом:

```asm
         mov cx,N

         jcxz Exit

   CYCL :

         <тело цикла>

         loop CYCL

   Exit :      . . .
```

После выхода из цикла содержимое регистра **ЕСХ/СХ** всегда равно нулю. 

#### Пример

```asm
%include "io.inc"

section .data  
n dd 10
str: db `Hello, World!\n`, 0

section .text
global CMAIN
CMAIN:
    mov ebp, esp; for correct debugging
    mov ecx, [n]
    xor ebx,ebx

    jcxz Exit

CYCL:
    inc ebx 
    loop CYCL

Exit:
    mov eax, 0
    ret
```

## Организация цикла с помощью команд loope / loopz.

Команды **loope** и **loopz** — абсолютные синонимы. Использовать можно любую из них.

Синтаксис команд:
```asm
loope <метка перехода>
loopz <метка перехода>
```

При использовании этих команд цикл повторяется до тех пор, пока **ЕСХ/СХ** не равно **0** и **ZF** равно **1**. Выход из цикла осуществляется при выполнении одного из двух условий: **ЕСХ/СХ** равно **0** и **ZF** равно **0**.

Работа команд заключается в выполнении следующих действий:

1. уменьшение значения регистра **ЕСХ/СХ** на 1;
2. сравнение регистра **ЕСХ/СХ** с нулем: если **ЕСХ/СХ** равно 0, осуществляется выход из цикла;
3. анализ состояния флага нуля **ZF**: если **ZF** равно 0, осуществляется выход из цикла.

Флаг нуля **ZF** принимает значение 0, когда сравниваемые операнды **не равны** между собой, и значение 1, если операнды **совпадают**. Для наглядности изобразим это схематически:
```asm
         cmp оп1, оп2

               оп1 = оп2 ⇒ zf=1

               оп1 ≠ оп2 ⇒ zf=0
```

Поэтому, используя этот флаг в качестве индикатора можно организовать досрочный (до того, как значение cx станет 0) выход из цикла.

Порядок организации цикла с помощью команды **loope**:

1. в регистр cx поместить значение, равное количеству повторений;
2. установить метку на первую команду тела цикла;
3. использовать команду cmp для сравнения операндов (здесь будет установлен флаг нуля);
4. выполнить команду ``loope <метка перехода>``.

Схема реализации выглядит следующим образом:
```asm
         mov cx,N
         jcxz Exit

   CYCL :
         <тело цикла>
         cmp <оп1>,<оп1>
         loope CYCL

    Exit: . . .
```

Так как флаг нуля принимает значение 0 в случае несовпадения операндов и, соответственно, выход из цикла осуществляется именно при несовпадении операндов, команды **loope/loopz удобно использовать для поиска первого элемента последовательности, отличного от заданной величины**.

#### Пример

```asm
%include "io.inc"

section .data  
n dd 10
str: db `Hello, World!\n`, 0

section .text
global CMAIN
CMAIN:
    mov ecx, [n]
    jcxz Exit
    mov eax, 10
    mov ebx, 1 
    mov edx, 15

CYCL:
    cmp eax, ebx
    jne ElsePart
    mov edx, ebx 
    jmp EndOfIf
ElsePart:
    inc ebx
EndOfIf:
    cmp edx, 15
    loope CYCL

Exit:
    mov eax, 0
    ret
```

## Организация цикла с помощью команд loopne/loopnz.

Команды **loopne/loopnz** являются обратными командам **loope/loopz**.
**Loopne** и **loopnz** также абсолютные синонимы, использовать можно любую из них.

Синтаксис команд:
```asm
Loopne <метка перехода>

loopnz <метка перехода>
```

При использовании этих команд цикл повторяется до тех пор, пока **ЕСХ/СХ** не равно **0** и **ZF** равно **0**.

Выход из цикла осуществляется при выполнении одного из двух условий: **ЕСХ/СХ** равно **0** и **ZF** равно **1**.

Работа команд заключается в выполнении следующих действий:

1. уменьшение значения регистра **ЕСХ/СХ** на 1;
2. сравнение регистра **ЕСХ/СХ** с нулем: если **ЕСХ/СХ** равно 0, осуществляется выход из цикла;
3. анализ состояния флага нуля **ZF**: если **ZF** равно 1, осуществляется выход из цикла.

Порядок организации цикла с помощью команды loopne :

1. в регистр cx поместить значение, равное количеству повторений;
2. установить метку на первую команду тела цикла;
3. использовать команду cmp для сравнения операндов (здесь будет установлен флаг нуля);
4. выполнить команду ``loopne <метка перехода>``.

Схема реализации выглядит следующим образом:
```asm
         mov cx , N
         jcxz Exit

   CYCL : 
         <тело цикла>
         cmp <оп1>,<оп1>
         loopne CYCL
   Exit:   . . .
```   

Так как флаг нуля принимает значение 1 в случае совпадения операндов и, соответственно, выход из цикла осуществляется именно при совпадении операндов, команды **loopne/loopnz можно использовать для поиска первого элемента последовательности, имеющего заданную величину**. 


#### Пример

```asm
%include "io.inc"

section .data  
n dd 10
str: db `Hello, World!\n`, 0

section .text
global CMAIN
CMAIN:
    mov ecx, [n]
    jcxz Exit
    mov eax, 10
    mov ebx, 1 
    mov edx, 5

CYCL:
    cmp eax, ebx
    jne ElsePart
    mov edx, ebx 
    jmp EndOfIf
ElsePart:
    inc ebx
EndOfIf:
    cmp edx, 10
    loopne CYCL

Exit:
    mov eax, 0
    ret
```

## Организация длинных циклов

Команды организации цикла loop , loope / loopz и loopne / loopnz , также как и команды условных переходов, реализуют только короткие переходы (от –128 до +127 байт), так как для адреса перехода в коде команд отводится только один байт. То есть тело цикла ограничивается только 128 байтами. Если тело цикла имеет большую длину, цикл необходимо по другому.

Команда безусловного перехода **jmp** позволяет осуществлять длинные переходы, так как в коде команды под смещение адреса перехода отводится два байта.

Для организации длинных циклов **следует использовать команды условного перехода и команду jmp**.

Схема реализации длинного цикла:
```asm
         mov cx,N

   CYCL:

         <тело цикла>

         dec cx

         cmp cx,0

         je Out

         jmp CYCL

   Out:    . . .
```

То есть, для организации длинного цикла необходимо «вручную» уменьшать содержимое параметра (счетчика) цикла и сравнивать полученное значение с нулем. Когда счетчик повторений становится равным нулю, с помощью команды условного перехода **je** следует выйти из цикла, иначе — командой безусловного перехода **jmp** необходимо осуществить переход на начало цикла.

### Цикл FOR

Типичная структура такого цикла на языке ассемблера:

```asm

      mov CX, n ;в cx помещаем количество повторений

for:  ………………

      ………………    ;тело цикла

      ………………

      dec CX

      cmp cx, 0

      jne for
```

Структура такого цикла на языке ассемблера с командой loop:

```asm

    mov cx, n  ;в cx помещаем количество повторений

m:  ………………

    ………………    ;тело цикла

    ………………

    loop m    ;cx уменьшается на 1 и, если он не равен 0, на метку m
```

#### Пример

```asm
%include "io.inc"

section .data  
n dd 10
str: db `Hello, World!\n`, 0

section .text
global CMAIN
CMAIN:
    mov ecx, [n]
    
FOR:    
    PRINT_STRING [str]  
    dec ecx
    cmp ecx,0
    jne FOR
    
EXIT:
    mov eax, 0
    ret
```

### Цикл WHILE

Типичная структура такого цикла WHILE на языке ассемблера:

```asm

A:  cmp cx, 0

    jle A2

    . . .   ;тело цикла

    jmp A

A2: . . . 
```
#### Пример

```asm
%include "io.inc"

section .data  
n dd 10
str: db `Hello, World!\n`, 0

section .text
global CMAIN
CMAIN:
    mov ecx, [n]
    
WHILE:
    cmp ecx,0
    jle EXIT
    PRINT_STRING [str]  
    dec ecx
    jmp WHILE
    
EXIT:
    mov eax, 0
    ret
```

### Цикл DO WHILE

Типичная структура такого цикла DO WHILE на языке ассемблера:

```asm

A:

;тело цикла

CMP X, 0

JGE A
```

#### Пример

```asm
%include "io.inc"

section .data  
n dd 10
str: db `Hello, World!\n`, 0

section .text
global CMAIN
CMAIN:
    mov ecx, [n]
    
DO_WHILE:
    PRINT_STRING [str]  
    dec ecx
    dec ecx
    dec ecx
    cmp ecx,0
    jge DO_WHILE
    
EXIT:
    mov eax, 0
    ret
```
 